In [43]:
from nltk.corpus import wordnet as wn
import numpy as np
from nltk import word_tokenize, pos_tag
import nltk
import pandas as pd
#nltk.download('averaged_perceptron_tagger')
#nltk.download('wordnet')
#nltk.download('punkt')
df = pd.read_csv('song.csv',header = None) 
df.columns = ['artist', 'song' ]

In [2]:
wn.synsets('do')

[Synset('bash.n.02'),
 Synset('do.n.02'),
 Synset('doctor_of_osteopathy.n.01'),
 Synset('make.v.01'),
 Synset('perform.v.01'),
 Synset('do.v.03'),
 Synset('do.v.04'),
 Synset('cause.v.01'),
 Synset('practice.v.01'),
 Synset('suffice.v.01'),
 Synset('do.v.08'),
 Synset('act.v.02'),
 Synset('serve.v.09'),
 Synset('do.v.11'),
 Synset('dress.v.16'),
 Synset('do.v.13')]

In [40]:



def penn_to_wn(tag):
    """ Convert between a Penn Treebank tag to a simplified Wordnet tag """
    if tag.startswith('N'):
        return 'n'
 
    if tag.startswith('V'):
        return 'v'
 
    if tag.startswith('J'):
        return 'a'
 
    if tag.startswith('R'):
        return 'r'
 
    return None
 
def tagged_to_synset(word, tag):
    wn_tag = penn_to_wn(tag)
    if wn_tag is None:
        return None
 
    try:
        return wn.synsets(word, wn_tag)[0]
    except:
        return None

def sentence_similarity(sentence1, sentence2):
    """ compute the sentence similarity using Wordnet """


    # Tokenize and tag
    sentence1 = pos_tag(word_tokenize(sentence1))
    sentence2 = pos_tag(word_tokenize(sentence2))
 
    # Get the synsets for the tagged words
    synsets1 = [tagged_to_synset(*tagged_word) for tagged_word in sentence1]
    synsets2 = [tagged_to_synset(*tagged_word) for tagged_word in sentence2]
 
    # Filter out the Nones
    synsets1 = [ss for ss in synsets1 if ss]
    synsets2 = [ss for ss in synsets2 if ss]
 
    score, count = 0.0, 0

 
    # For each word in the first sentence
    #for synset in synsets1:
        # Get the similarity value of the most similar word in the other sentence
        #best_score = max([synset.path_similarity(ss) for ss in synsets2])
    arr_simi_score = []
    for syn1 in synsets1:
        for syn2 in synsets2:
            simi_score = syn1.wup_similarity(syn2)

 
        # Check that the similarity could have been computed
            if simi_score is not None:
                arr_simi_score.append(simi_score)
                best = max(arr_simi_score)
                score += best
                count += 1
            #if best is not None:
            #    score += best
             #   count += 1
 
    # Average the values
    if count != 0:
        score /= count


    return score

def symmetric_sentence_similarity(sentence1, sentence2):
    """ compute the symmetric sentence similarity using Wordnet """
    return (sentence_similarity(sentence1, sentence2) + sentence_similarity(sentence2, sentence1)) / 2

sentences = []
for x in range(1000):
    sentences = sentences + [df['song'][x]]
 

    #print ("Similarity(\"%s\", \"%s\") = %s" % (sentence, focus_sentence, sentence_similarity(sentence, focus_sentence)))
    

# similarity("Cats are beautiful animals.", "Dogs are awesome.") = 0.511111111111
# Similarity("Dogs are awesome.", "Cats are beautiful animals.") = 0.666666666667
 
# Similarity("Cats are beautiful animals.", "Some gorgeous creatures are felines.") = 0.833333333333
# Similarity("Some gorgeous creatures are felines.", "Cats are beautiful animals.") = 0.833333333333
 
# Similarity("Cats are beautiful animals.", "Dolphins are swimming mammals.") = 0.483333333333
# Similarity("Dolphins are swimming mammals.", "Cats are beautiful animals.") = 0.4
 
# Similarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0
# Similarity("Cats are beautiful animals.", "Cats are beautiful animals.") = 1.0

In [72]:
def last_search(query):
    focus_sentence = query
    rank = []
    result = ()
    result2 = ()
    index = 0
    floatrank = np.asarray(rank, dtype = np.float32)
    floatrank2 = np.asarray(rank, dtype = np.float32)
    for sentence in sentences:
        similarity = symmetric_sentence_similarity(focus_sentence, sentence)
        #print ("Similarity(\"%s\", \"%s\") = %s" % (focus_sentence, sentence, similarity))
        if similarity > 0.8:
            result = result + (sentence, similarity)
            floatrank = np.asarray(similarity)
            floatrank2 =  np.append(floatrank2, floatrank)
            result2 = result2 + (index,)
        index += 1
    print(result)
    print(result2)
    for x in range(len(result2)):
        print(df['artist'][(result2[x])] + " " + df['song'][(result2[x])])

In [74]:
last_search('light')

('Green Light', 1.0, 'Moonlight', 0.9565217391304348, 'Moonlight', 0.9565217391304348, 'New Light', 1.0, 'All of the Lights', 1.0, 'Starlight', 0.9565217391304348, 'The Light Is Coming (feat. Nicki Minaj)', 1.0, 'Sunshine of Your Love', 0.9565217391304348, 'Light On', 1.0)
(114, 193, 220, 378, 390, 595, 717, 737, 955)
Lorde Green Light
Foals Moonlight
xxxtentacion Moonlight
John Mayer New Light
Kanye West All of the Lights
Muse Starlight
Ariana Grande The Light Is Coming (feat. Nicki Minaj)
Cream Sunshine of Your Love
Maggie Rogers Light On


In [46]:
df['artist'][1]

'Ariana Grande'